### Imports

In [1]:

import re
import os
import json
import pickle
import shutil
import requests
import warnings
from collections import defaultdict


import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import warnings
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error, mean_absolute_error


### Starter code

In [2]:
class SmartBuildingsDataset:
 """Smart Buildings Dataset implementation, including loading and downloading."""


 def __init__(self, download=True):
   self.partitions = {
       "sb1": [
           "2022_a",
           "2022_b",
           "2023_a",
           "2023_b",
           "2024_a",
       ],
   }
   if download:
     self.download()


 def download(self):
   """Downloads the Smart Buildings Dataset from Google Cloud Storage."""
   print("Downloading data...")


   def download_file(url):
     local_filename = url.split("/")[-1]
     with requests.get(url, stream=True) as r:
       r.raise_for_status()
       with open(local_filename, "wb") as f:
         for chunk in r.iter_content(chunk_size=8192):
           f.write(chunk)
     return local_filename


   url = "https://storage.googleapis.com/gresearch/smart_buildings_dataset/tabular_data/sb1.zip"
   download_file(url)
   shutil.unpack_archive("sb1.zip", "sb1/")


 def get_floorplan(self, building):
   """Gets the floorplan and device layout map for a specific building.


   Args:
     building: The name of the building.


   Returns:
     A tuple containing the floorplan and device layout map.
   """
   if building not in self.partitions.keys():
     raise ValueError("invalid building")
   floorplan = np.load(f"./{building}/tabular/floorplan.npy")
   with open(f"./{building}/tabular/device_layout_map.json") as json_file:
     device_layout_map = json.load(json_file)
   return floorplan, device_layout_map


 def get_building_data(self, building, partition):
   """Gets the data for a specific building and partition.


   Args:
     building: The name of the building.
     partition: The name of the partition.


   Returns:
     A tuple containing the data and metadata.
   """
   if building not in self.partitions.keys():
     raise ValueError("invalid building")
   if partition not in self.partitions[building]:
     raise ValueError("invalid partition")
   path = f"./{building}/tabular/{building}/{partition}/"


   data = np.load(path + "data.npy.npz")
   metadata = pickle.load(open(path + "metadata.pickle", "rb"))


   if "device_infos" not in metadata.keys():
     metadata["device_infos"] = pickle.load(
         open(f"./{building}/tabular/device_info_dicts.pickle", "rb")
     )
   if "zone_infos" not in metadata.keys():
     metadata["zone_infos"] = pickle.load(
         open(f"./{building}/tabular/zone_info_dicts.pickle", "rb")
     )
   return data, metadata


In [3]:
ds = SmartBuildingsDataset()

# training data: Jan-June 2022
data, metadata = ds.get_building_data("sb1","2022_a")
floorplan, device_layout_map = ds.get_floorplan("sb1")

# validation data: July-December 2022
data_val, metadata_val = ds.get_building_data("sb1","2022_b")
floorplan_val, device_layout_map_val = ds.get_floorplan("sb1")

# lets split validation data into things to predict, and exogenous variables
indexes = [v for k, v in metadata_val['observation_ids'].items() if "zone_air_temperature_sensor" in k]
temp_data = data_val['observation_value_matrix'][:, indexes]
matching_items = [(k, v) for k, v in metadata_val['observation_ids'].items() if "zone_air_temperature_sensor" in k]
temp_data_ids = {k: i for i, (k, v) in enumerate(matching_items)}

indexes = [v for k, v in metadata_val['observation_ids'].items() if "zone_air_temperature_sensor" not in k]
exogenous_observation_data = data_val['observation_value_matrix'][:, indexes]
matching_items = [(k, v) for k, v in metadata_val['observation_ids'].items() if "zone_air_temperature_sensor" not in k]
exogenous_observation_data_ids = {k: i for i, (k, v) in enumerate(matching_items)}

initial_condition = temp_data[0]
print(len(data["observation_value_matrix"]))

51852


In [4]:
data2 = dict(data)
metadata2 = dict(metadata)

data2["observation_value_matrix"] = np.concatenate(
     [data["observation_value_matrix"], data_val["observation_value_matrix"]], axis=0
 )
data2["action_value_matrix"] = np.concatenate(
     [data["action_value_matrix"], data_val["action_value_matrix"]], axis=0
 )
metadata2["observation_timestamps"] = np.concatenate(
     [metadata["observation_timestamps"], metadata_val["observation_timestamps"]], axis=0
 )
metadata2["action_timestamps"] = np.concatenate(
     [metadata["action_timestamps"], metadata_val["action_timestamps"]], axis=0
 )

data, metadata = data2, metadata2
data, metadata = dict(data), dict(metadata)

In [5]:
# Train data:
# data['observation_value_matrix']
# data['action_value_matrix']
# metadata["observation_ids"]
# metadata["action_ids"]
# metadata["observation_timestamps"]
# floorplan
# device_layout_map


# Validation data:
# data_val['action_value_matrix']
# metadata_val["action_ids"]
# metadata_val["observation_timestamps"]
# floorplan
# device_layout_map

# exogenous_observation_data
# exogenous_observation_data_ids
# initial_condition

# Predict:
# temp_data
# temp_data_ids

### Prepare data

In the original dataset, the timestamps for `reward`and `reward_info` begin (and end) one timestep earlier than the timestamps for `observation` and `action`. Let's align these for convenience.

In [6]:
data, metadata = dict(data), dict(metadata) # convert to dicts for convenience

# these start one step earlier, so we need to remove the first one
data["reward_value_matrix"] = data["reward_value_matrix"][1:]
data["reward_info_value_matrix"] = data["reward_info_value_matrix"][1:]
metadata["reward_timestamps"] = metadata["reward_timestamps"][1:]
metadata["reward_info_timestamps"] = metadata["reward_info_timestamps"][1:]

# these end one step later, so we need to remove the last one
data["observation_value_matrix"] = data["observation_value_matrix"][:-1]
data["action_value_matrix"] = data["action_value_matrix"][:-1]
metadata["observation_timestamps"] = metadata["observation_timestamps"][:-1]
metadata["action_timestamps"] = metadata["action_timestamps"][:-1]

Let's create a dataframe to manipulate the data. We will include the data from `observation` and `action`. We disregard the reward data because intuitively, I imagine that is more important for training some RL agent, rather than predicting the temperature.

We add a `device_id` column to tag each row with the device it belongs to.

In [7]:
df_obs_ts = pd.DataFrame(
    data["observation_value_matrix"],
    columns=list(metadata["observation_ids"].keys()),
    index=metadata["observation_timestamps"], # Use the aligned timestamps
)

df_act_ts = pd.DataFrame(
    data["action_value_matrix"],
    columns=list(metadata["action_ids"].keys()),
    index=metadata["action_timestamps"], # Use the aligned timestamps
)

df_combined_ts = pd.concat([df_obs_ts, df_act_ts], axis=1)
df = df_combined_ts.T

# add device_id column
df["device_id"] = df.index.str.split("@").str[0]

Now, let's create mappings from `device_id` to `zone_id` and vice-versa.

To decide which observations to use for each device, we will use the `zone_id` to which the device belongs. Create these mappings to make it easier to access the data.

In [8]:
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

zone_infos = { el["zone_id"]: el for el in metadata["zone_infos"] }

# device_to_zones
device_to_zones = defaultdict(lambda: [])
for zone_id, zone_info in zone_infos.items():
    for device_id in zone_info["devices"]:
        device_to_zones[device_id].append(zone_id)
        
zone_to_devices = defaultdict(lambda: [])
for zone_id, zone_info in zone_infos.items():
    for device_id in zone_info["devices"]:
        zone_to_devices[zone_id].append(device_id)
        
# add mapping information to dataframe
for zone_id, zone_info in zone_infos.items():
    df[zone_id] = df["device_id"].apply(lambda x: zone_id in device_to_zones[x])

Now, let's also get all the devices which have `zone_air_temperature_sensor` data. These are the values that we are trying to predict.

In [9]:
# get all devices with temperature measurements
devices_with_temp = [k.split('@')[0] for k, v in metadata["observation_ids"].items() if "zone_air_temperature_sensor" in k]

### Feature selection logic

The idea we will use is the following:
1. We will try to predict the temperature of each sensor individually.
2. For each device, we will use the `device_info` data to get all the `zone_id`'s that are associated with each device.
3. Then, we will get all the observations and actions for devices that are associated with any of the `zone_id`'s above.
4. We will use these observations and actions as features to predict the temperature of this sensor.

Basically, the logic here is: devices that are associated with the same zones should influence each other. So using the observations and actions of these devices hopefully will be predictive of the temperature of the sensor we are trying to predict.

For each sensor, we will make a personalized dataframe with the measurements which will be used to train this sensor's model. The function below implements this logic.

In [10]:
def get_filtered_df_for_device(df, device_id):
    zones_ids = device_to_zones[device_id]
    
    is_timestamp_col = [isinstance(col, pd.Timestamp) for col in df.columns]
    is_timestamp_mask = pd.Series(is_timestamp_col, index=df.columns)
    zone_mask = df.columns.isin(zones_ids)
    
    rows_in_common_zones_mask = df.loc[:, zone_mask].sum(axis=1) > 0
    
    filtered_df = df.loc[rows_in_common_zones_mask, :]
    
    obs_df = filtered_df.loc[~filtered_df.index.str.contains("zone_air_temperature_sensor"), is_timestamp_mask]
    target_df = filtered_df.loc[filtered_df.index.str.contains(f"{ device_id }@zone_air_temperature_sensor"), is_timestamp_mask]

    return obs_df, target_df

We also add features to provide information about the time of the day, day of the week, etc. This is useful because the temperature is very dependent on these, as the building behaves differently at different times of the day and week (e.g. weekdays vs weekends).

In [11]:
def add_time_features(dataframe):
    """Adds time features to the dataframe. Index should be timestamps."""
    copy_df = dataframe.copy()
    copy_df.index = pd.to_datetime(copy_df.index, errors='raise')
    copy_df['time_year'] = copy_df.index.year
    copy_df['time_month'] = copy_df.index.month
    copy_df['time_day'] = copy_df.index.day
    copy_df['time_hour'] = copy_df.index.hour
    copy_df['time_minute'] = copy_df.index.minute

    return copy_df

#### Training helper functions

In [12]:
def save_results(timestamps, device_id, rmse_xgb, mae_xgb, rmse_mean, mae_mean, y_val, y_pred_xgb, output_dir):
    """
    Saves model performance metrics to JSON and generates two plots:
    1. A scatter plot of actual vs. predicted values.
    2. A time-series line plot of actual and predicted values using provided timestamps.

    Args:
        timestamps (array-like): The timestamps corresponding to the validation data points for the x-axis.
        device_id (str or int): Identifier for the device/model.
        rmse_xgb (float): Root Mean Squared Error for the XGBoost model.
        mae_xgb (float): Mean Absolute Error for the XGBoost model.
        rmse_mean (float): Root Mean Squared Error for the baseline mean model.
        mae_mean (float): Mean Absolute Error for the baseline mean model.
        y_val (np.ndarray): True target values for the validation set.
        y_pred_xgb (np.ndarray): Predicted target values from the XGBoost model.
        output_dir (str): The base directory where results should be saved.
    """
    # Create device-specific directory
    device_output_dir = os.path.join(output_dir, f"device_{device_id}")
    os.makedirs(device_output_dir, exist_ok=True) # Create directory if it doesn't exist

    # --- 1. Save Metrics to JSON ---
    results_data = {
        'device_id': device_id,
        'performance_metrics': {
            'xgboost': {
                'rmse': rmse_xgb,
                'mae': mae_xgb
            },
            'baseline_mean': {
                'rmse': rmse_mean,
                'mae': mae_mean
            }
        },
        'validation_set_size': len(y_val)
    }

    json_filepath = os.path.join(device_output_dir, 'results.json')
    try:
        with open(json_filepath, 'w') as f:
            json.dump(results_data, f, indent=4)
        print(f"Device {device_id}: Saved results to {json_filepath}")
    except Exception as e:
        print(f"Device {device_id}: Error saving JSON results: {e}")


    # --- 2. Generate and Save Scatter Plot (Actual vs. Predicted) ---
    scatter_plot_filepath = os.path.join(device_output_dir, 'predictions_vs_actual.png')
    try:
        plt.figure(figsize=(8, 8))
        plt.scatter(y_val, y_pred_xgb, alpha=0.5, label=f'XGB Predictions (RMSE: {rmse_xgb:.2f})') # Keep scatter for this plot
        min_val = min(np.min(y_val), np.min(y_pred_xgb)) * 0.95
        max_val = max(np.max(y_val), np.max(y_pred_xgb)) * 1.05
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect Prediction') # Keep diagonal line
        plt.title(f'Device {device_id}: Actual vs. Predicted Values')
        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.legend()
        plt.grid(True)
        plt.xlim(min_val, max_val)
        plt.ylim(min_val, max_val)
        plt.savefig(scatter_plot_filepath)
        plt.close() # Close the plot figure to free memory
        print(f"Device {device_id}: Saved scatter plot to {scatter_plot_filepath}")
    except Exception as e:
        print(f"Device {device_id}: Error saving scatter plot: {e}")


    # --- 3. Generate and Save Time Series Plot ---
    # Uses provided timestamps for the x-axis.
    timeseries_plot_filepath = os.path.join(device_output_dir, 'predictions_over_time.png')
    try:
        # Use the provided timestamps directly
        time_indices = timestamps

        plt.figure(figsize=(12, 6)) # Make the figure wider for time series

        # Plot actual values as a navy line
        plt.plot(time_indices, y_val, label='Actual Values', color='navy', linestyle='-', linewidth=1.5)

        # Plot predicted values as a crimson line
        plt.plot(time_indices, y_pred_xgb, label='Predicted Values', color='crimson', linestyle='-', linewidth=1.5)

        plt.title(f'Device {device_id}: Actual vs. Predicted Values Over Time')
        # Update xlabel if timestamps are actual dates/times, otherwise keep as is
        # If timestamps are datetime objects, matplotlib will usually format them automatically.
        plt.xlabel('Time') # Changed label to be more generic
        plt.ylabel('Value')
        plt.legend()
        plt.grid(True)
        plt.tight_layout() # Adjust layout
        # Optional: Add automatic date formatting if timestamps are datetime objects
        # plt.gcf().autofmt_xdate() # Uncomment if time_indices contains datetime objects

        plt.savefig(timeseries_plot_filepath)
        plt.close() # Close the plot figure to free memory
        print(f"Device {device_id}: Saved time series plot to {timeseries_plot_filepath}")
    except Exception as e:
        print(f"Device {device_id}: Error saving time series plot: {e}")

### Training

Now, to train the model, we will do the following:

1. We will use the features that were selected according to the logic of the previous section.
2. For each of the selected features, we will also create lagged versions of them. This is because we understand that previous observations and actions should be predictive of the current temperature.
3. We pre-process the features using a `StandardScaler`.

In [13]:
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings('ignore', category=UserWarning) # Suppress XGBoost warning about feature names

scalers, models, processed_feature_names = {}, {}, {}

def get_train_val_data_for_device(device_id, lags_exo=[1, 2, 6, 12, 24]):
    
    X_base, y_base = get_filtered_df_for_device(df, device_id)
    X_base, y_base = X_base.T, y_base.T
    X_base, y_base = add_time_features(X_base), add_time_features(y_base)
    
    y_series = y_base.iloc[:, 0]
    
    temp_df = X_base.copy()
    temp_df["target"] = y_series
    temp_df = temp_df.apply(pd.to_numeric, errors='raise')

    
    lagged_target_cols = []
    lagged_exo_cols = []
    
    
    # add lag features
    exo_cols_to_lag = list(X_base.columns)
    for col in exo_cols_to_lag:
        
        # this should not happen, but just in case
        if col not in temp_df.columns:
            raise ValueError(f"Column {col} not found in temp_df for lagging.")
         
        for lag in lags_exo:
                col_name = f'{str(col).replace("@","_")}_lag_{lag}' # Make lag name safe
                temp_df[col_name] = temp_df[col].shift(lag)
                lagged_exo_cols.append(col_name)
                
    
    # filter out timestamps for which the target is 0 (missing data)
    original_rows = len(temp_df)
    temp_df = temp_df[temp_df['target'] != 0]
    rows_after_filtering = len(temp_df)
    print(f"Filtered {original_rows - rows_after_filtering} rows for device {device_id}. {rows_after_filtering} rows remaining.")
    
    # drop rows with NaN values due to lagging
    temp_df.dropna(inplace=True)
    
    X = temp_df.drop(columns=["target"])
    y = temp_df["target"]
    target_col_name = f"{ device_id }@zone_air_temperature_sensor"
    processed_feature_names[target_col_name] = X.columns.tolist()
    
    cols_to_scale = list(X_base.columns) + lagged_exo_cols + lagged_target_cols
    scaler = StandardScaler()
    X_scaled = X.copy()
    X_scaled[cols_to_scale] = scaler.fit_transform(X[cols_to_scale])
    scalers[target_col_name] = scaler
    
    
    # # split into train and validation sets
    # train_end_index = int(len(X_scaled) * 0.7) # 70% train

    # X_train, X_val = X_scaled[:train_end_index], X_scaled[train_end_index:]
    # y_train, y_val = y[:train_end_index], y[train_end_index:]
    
    split_timestamp = pd.Timestamp('2022-07-01 00:00:00+0000', tz='UTC')

    # Create boolean masks for train and validation sets based on the index
    # Ensure X_scaled.index is a DatetimeIndex before this step
    train_mask = X_scaled.index < split_timestamp
    val_mask = X_scaled.index >= split_timestamp

    # Apply the masks to split the data using .loc
    X_train, X_val = X_scaled.loc[train_mask], X_scaled.loc[val_mask]
    y_train, y_val = y.loc[train_mask], y.loc[val_mask] # Assumes y shares the same DatetimeIndex as X_scaled
    # -------- END OF LINES TO ADD --------
    
    print(f"Device {device_id}: Train set size: {len(X_train)}, Validation set size: {len(X_val)}")
    
    return X_train, y_train, X_val, y_val

To train the model, we use a `XGBoost regressor`. I haven't done any experimentation with other models/hyperperameters due to time constraints, so hopefully this is a good starting point.

In [14]:
def train_and_eval(X_train, y_train, X_val, y_val, device_id):
    """
    Train and evaluate the model.
    """
    
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=300,          # Potentially more estimators needed with more features
        learning_rate=0.05,
        max_depth=7,
        subsample=0.8,
        colsample_bytree=0.8,      # Consider slightly lower if many features added
        random_state=42,
        n_jobs=-1,
        early_stopping_rounds=30   # Potentially longer patience
    )
    
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )


    y_pred_xgb = model.predict(X_val)
    rmse_xgb = root_mean_squared_error(y_val, y_pred_xgb) # Use RMSE directly
    mae_xgb = mean_absolute_error(y_val, y_pred_xgb)
    
    
    mean_train_value = y_train.mean()
    y_pred_mean = np.full_like(y_val, fill_value=mean_train_value)
    rmse_mean = root_mean_squared_error(y_val, y_pred_mean)
    mae_mean = mean_absolute_error(y_val, y_pred_mean)
    
    models[device_id] = model
    
    save_results(list(X_val.index), device_id, rmse_xgb, mae_xgb, rmse_mean, mae_mean, y_val, y_pred_xgb, "results_fr")
    
    return

Now, let's actually train the models.

In [15]:
for device_id in tqdm(devices_with_temp):
    X_train, y_train, X_val, y_val = get_train_val_data_for_device(device_id)
    train_and_eval(X_train, y_train, X_val, y_val, device_id)

  0%|          | 0/123 [00:00<?, ?it/s]

Filtered 20739 rows for device 2760348383893915. 84404 rows remaining.
Device 2760348383893915: Train set size: 36642, Validation set size: 47738
Device 2760348383893915: Saved results to results_fr/device_2760348383893915/results.json
Device 2760348383893915: Saved scatter plot to results_fr/device_2760348383893915/predictions_vs_actual.png


  1%|          | 1/123 [00:21<43:30, 21.40s/it]

Device 2760348383893915: Saved time series plot to results_fr/device_2760348383893915/predictions_over_time.png
Filtered 20221 rows for device 2562701969438717. 84922 rows remaining.
Device 2562701969438717: Train set size: 37010, Validation set size: 47888
Device 2562701969438717: Saved results to results_fr/device_2562701969438717/results.json
Device 2562701969438717: Saved scatter plot to results_fr/device_2562701969438717/predictions_vs_actual.png


  2%|▏         | 2/123 [00:23<20:12, 10.02s/it]

...

Device 2559962019321287: Saved time series plot to results_fr/device_2559962019321287/predictions_over_time.png
Filtered 19939 rows for device 2606839440408366. 85204 rows remaining.
Device 2606839440408366: Train set size: 37224, Validation set size: 47956
Device 2606839440408366: Saved results to results_fr/device_2606839440408366/results.json
Device 2606839440408366: Saved scatter plot to results_fr/device_2606839440408366/predictions_vs_actual.png


 99%|█████████▉| 122/123 [46:22<00:12, 12.98s/it]

Device 2606839440408366: Saved time series plot to results_fr/device_2606839440408366/predictions_over_time.png
Filtered 20489 rows for device 2640423556868160. 84654 rows remaining.
Device 2640423556868160: Train set size: 36943, Validation set size: 47687
Device 2640423556868160: Saved results to results_fr/device_2640423556868160/results.json
Device 2640423556868160: Saved scatter plot to results_fr/device_2640423556868160/predictions_vs_actual.png


100%|██████████| 123/123 [46:26<00:00, 22.66s/it]

Device 2640423556868160: Saved time series plot to results_fr/device_2640423556868160/predictions_over_time.png


In [16]:
# %% [markdown]
# ### Calculate Average Performance Metrics

# %%

# --- Code to add at the end of your script ---

results_dir = "results_fr" # The directory specified in save_results
all_devices = devices_with_temp # Get the list of devices that were processed

# Initialize lists to store metrics from each device
rmse_xgb_list = []
mae_xgb_list = []
rmse_mean_list = []
mae_mean_list = []

print(f"\nReading results from '{results_dir}' to calculate average metrics...")
print(f"Processing results for {len(all_devices)} devices.")

# Iterate through all devices for which models were trained
for device_id in all_devices:
    # Construct the path to the results JSON file for the current device
    device_output_dir = os.path.join(results_dir, f"device_{device_id}")
    json_filepath = os.path.join(device_output_dir, 'results.json')

    # Check if the results file exists
    if os.path.exists(json_filepath):
        try:
            # Open and load the JSON data
            with open(json_filepath, 'r') as f:
                results_data = json.load(f)

            # Extract the performance metrics dictionary
            metrics = results_data.get('performance_metrics', {})
            xgb_metrics = metrics.get('xgboost', {})
            mean_metrics = metrics.get('baseline_mean', {})

            # Append metrics to the lists if they exist
            # Check specifically for the keys to avoid errors if a file is incomplete
            if 'rmse' in xgb_metrics:
                rmse_xgb_list.append(xgb_metrics['rmse'])
            else:
                print(f"Warning: Missing XGBoost RMSE for device {device_id}")

            if 'mae' in xgb_metrics:
                mae_xgb_list.append(xgb_metrics['mae'])
            else:
                print(f"Warning: Missing XGBoost MAE for device {device_id}")

            if 'rmse' in mean_metrics:
                rmse_mean_list.append(mean_metrics['rmse'])
            else:
                 print(f"Warning: Missing Baseline Mean RMSE for device {device_id}")

            if 'mae' in mean_metrics:
                mae_mean_list.append(mean_metrics['mae'])
            else:
                 print(f"Warning: Missing Baseline Mean MAE for device {device_id}")


        except json.JSONDecodeError:
            print(f"Warning: Could not decode JSON file for device {device_id} at {json_filepath}. Skipping.")
        except Exception as e:
            print(f"Warning: Error processing results file for device {device_id}: {e}. Skipping.")
    else:
        # This case might happen if save_results failed for some reason
        print(f"Warning: Results file not found for device {device_id} at {json_filepath}. Skipping.")

# Calculate the averages, ensuring we don't divide by zero
# Check if we actually collected any valid results
num_valid_results = len(rmse_xgb_list) # Assuming all lists should have same length if valid

if num_valid_results > 0:
    # Calculate simple arithmetic average
    avg_rmse_xgb = sum(rmse_xgb_list) / num_valid_results
    avg_mae_xgb = sum(mae_xgb_list) / num_valid_results
    avg_rmse_mean = sum(rmse_mean_list) / num_valid_results # Should match num_valid_results if data is consistent
    avg_mae_mean = sum(mae_mean_list) / num_valid_results   # Should match num_valid_results if data is consistent

    print("\n--- Average Performance Metrics Across All Devices ---")
    print(f"(Based on {num_valid_results} devices with valid results files)")
    print(f"\nBaseline (Mean Prediction):")
    print(f"  Average RMSE: {avg_rmse_mean:.4f}")
    print(f"  Average MAE:  {avg_mae_mean:.4f}")
    print(f"\nXGBoost Models:")
    print(f"  Average RMSE: {avg_rmse_xgb:.4f}")
    print(f"  Average MAE:  {avg_mae_xgb:.4f}")

else:
    print("\nCould not calculate average metrics: No valid results files were found or processed.")

# --- End of code to add ---


Reading results from 'results_fr' to calculate average metrics...
Processing results for 123 devices.

--- Average Performance Metrics Across All Devices ---
(Based on 123 devices with valid results files)

Baseline (Mean Prediction):
  Average RMSE: 3.3639
  Average MAE:  2.6695

XGBoost Models:
  Average RMSE: 2.5347
  Average MAE:  1.7434
